In [1]:
# Librerías de manejo de datos
import pandas as pd
import numpy as np

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.7f' % x)

# Librerías de análisis técnico y estadística
import talib as ta
from scipy.stats import pointbiserialr
from sklearn.feature_selection import mutual_info_regression

# Librerías de paralelización y optimización
import joblib
from joblib import Parallel, delayed, parallel_backend
import cupy as cp

# Manejo de fechas y tiempo
import time
from datetime import datetime

# Librerías de bases de datos y almacenamiento
import os
import pickle
import hashlib

# Utilidades varias
import uuid
import itertools
import re
import gc
from tqdm import tqdm


In [2]:


def transform_df(csv_name, exposicion_dias=3, threshold=25, date_column='DateTime', short=False):
    df = pd.read_csv(csv_name+'.csv')

    def rsi_function(i):
        rsi = ta.RSI(df['Close'], timeperiod=i)
        return pd.DataFrame(rsi, columns=[f'rsi_{i}'])

    rsi_dfs = pd.concat([rsi_function(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, rsi_dfs], axis=1)

    def adx_function(i):
        adx = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=i)
        return pd.DataFrame(adx, columns=[f'adx_{i}'])

    adx_dfs = pd.concat([adx_function(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, adx_dfs], axis=1)

    def plus_di_function(i):
        plus_di = ta.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod=i)
        return pd.DataFrame(plus_di, columns=[f'plus_di_{i}'])

    plus_di_dfs = pd.concat([plus_di_function(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, plus_di_dfs], axis=1)

    def minus_di_function(i):
        minus_di = ta.MINUS_DI(df['High'], df['Low'], df['Close'], timeperiod=i)
        return pd.DataFrame(minus_di, columns=[f'minus_di_{i}'])

    minus_di_dfs = pd.concat([minus_di_function(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, minus_di_dfs], axis=1)

    def willr_function(i):
        willr = ta.WILLR(df['High'], df['Low'], df['Close'], timeperiod=i)
        return pd.DataFrame(willr, columns=[f'willr_{i}'])

    willr_dfs = pd.concat([willr_function(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, willr_dfs], axis=1)

    def ma_function(i):
        ma = ta.MA(df['Close'], timeperiod=i, matype=0)
        return pd.DataFrame(ma, columns=[f'sma_{i}'])

    ma_dfs = pd.concat([ma_function(i) for i in range(2, 301, 2)], axis=1)
    df = pd.concat([df, ma_dfs], axis=1)

    def ema_function(i):
        ma = ta.EMA(df['Close'], timeperiod=i)
        return pd.DataFrame(ma, columns=[f'mema_{i}'])

    ema_dfs = pd.concat([ema_function(i) for i in range(2, 301, 2)], axis=1)
    df = pd.concat([df, ema_dfs], axis=1)

    def atr_function(i):
        atr = ta.ATR(df['High'], df['Low'], df['Close'], timeperiod=i)
        return pd.DataFrame(atr, columns=[f'atr_{i}'])

    atr_dfs = pd.concat([atr_function(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, atr_dfs], axis=1)

    def calculate_ibs(high, low, close):
        ibs = (close - low) / (high - low)
        ibs = np.round(ibs, 2)
        return pd.DataFrame(ibs, columns=['ibs_'])


    def stdev_function(i):
            stdev = ta.STDDEV(df['Close'], timeperiod=i, nbdev=1)
            return pd.DataFrame(stdev, columns=[f'stdev_{i}'])
        
    stdev_dfs = pd.concat([stdev_function(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, stdev_dfs], axis=1)

    def bband_function(i, dev=2):
        upperband, middleband, lowerband = ta.BBANDS(df['Close'], timeperiod=i, nbdevup=dev, nbdevdn=dev, matype=0)
        return pd.DataFrame({f'bb_upper_{dev}_{i}': upperband, f'bb_middle_{dev}_{i}': middleband, f'bb_lower_{dev}_{i}': lowerband})
    
    def bband_function(i, dev=2):
        upperband, middleband, lowerband = ta.BBANDS(df['Close'], timeperiod=i, nbdevup=dev, nbdevdn=dev, matype=0)
        return pd.DataFrame({f'bb_upper_{dev}_{i}': upperband, f'bb_lower_{dev}_{i}': lowerband})
	
    for dev in range(2,6):
        bband_dfs = pd.concat([bband_function(i, dev) for i in range(5, 31, 2)], axis=1)
        df = pd.concat([df, bband_dfs], axis=1)
        

    def macd_function(fp, slp, sp):
        macd, macdsignal, macdhist = ta.MACD(df['Close'], fastperiod=fp, slowperiod=slp, signalperiod=sp)
        return pd.DataFrame({f'macd_{fp}': macd, f'macdsig_{slp}': macdsignal, f'macdh_{sp}': macdhist})

    macd_dfs = []
    fastperiod_values = [7, 12, 26, 52]
    slowperiod_values = [13, 26, 52]
    signalperiod_values = [3, 6, 9]


    def mom_function(i):
        momentum = ta.MOM(df['Close'], timeperiod=i)
        return pd.DataFrame(momentum, columns=[f'mom_{i}'])

    momentum_dfs = pd.concat([mom_function(i) for i in range(2, 31, 2)], axis=1)
    df = pd.concat([df, momentum_dfs], axis=1)

    def aaron_up_function(i):
        aroon_up = ta.AROONOSC(df['High'], df['Low'], timeperiod=i)
        return pd.DataFrame(aroon_up, columns=[f'aaro_{i}'])

    aaronu_dfs = pd.concat([aaron_up_function(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, aaronu_dfs], axis=1)
	
    def aaron_up_function2(i):
        _, aroon_up = ta.AROON(df['High'], df['Low'], timeperiod=i)
        return pd.DataFrame(aroon_up, columns=[f'aarou_{i}'])
	
    aaronu_dfs = pd.concat([aaron_up_function2(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, aaronu_dfs], axis=1)
	
    def aaron_dw_function2(i):
        aroon_down, _ = ta.AROON(df['High'], df['Low'], timeperiod=i)
        return pd.DataFrame(aroon_down, columns=[f'aarod_{i}'])
	
    aaronu_dfs = pd.concat([aaron_dw_function2(i) for i in range(2, 51, 2)], axis=1)
    df = pd.concat([df, aaronu_dfs], axis=1)

    def roc_function(i):
        roc_t = ta.ROC(df['Close'], timeperiod=i)
        return pd.DataFrame(roc_t, columns=[f'roc_{i}'])




    duplicates = df.columns[df.columns.duplicated()]
    df = df.loc[:, ~df.columns.duplicated()]

    def shift_column(column, i):
        shifted = df[column].shift(i)
        if('ibs_' in column):
            column = 'ibs'
        return shifted.rename(f'{column}_sft_{i}')

    columns = df.columns
    lista_shift = ['rsi', 'adx', 'plus_di', 'minus_di', 'willr', 'bb', 'atr', 'stdev', 'Close', 'High', 'Low', 'aaro', 'mom']
    indicator_columns = {col for col in columns if any(name in col for name in lista_shift)}

    shifted_columns = []
    shift_value = 3

    for column in indicator_columns:
        for i in range(1, shift_value + 1):
            shifted_series = shift_column(column, i)
            shifted_columns.append(shifted_series)

    df = pd.concat([df] + shifted_columns, axis=1)


    pips = 0
    if "JPY" in csv_name:
        pips = 100
    else:
        pips = 10000


    for i in range (2,31, 2):
        ret = []
        new_cols = []
        if(short == True):
            ret = ((df["Close"].shift(-1 * i) - df["Close"]) * pips) + 2
            new_cols = pd.DataFrame(np.array(ret) * -1, columns=[f"Return_{i}"])
        else:
            ret = ((df["Close"].shift(-1 * i) - df["Close"]) * pips) - 2
            new_cols = pd.DataFrame(np.array(ret), columns=[f"Return_{i}"])
			
        df = pd.concat([df, new_cols], axis=1)


    if(short == True):
        ret = ((df["Close"].shift(-1 * exposicion_dias) - df["Close"]) * pips) + 2
    else:
        ret = ((df["Close"].shift(-1 * exposicion_dias) - df["Close"]) * pips) - 2
		
    new_cols = pd.DataFrame(np.array(ret), columns=["Return"])
	
    if(short == True):
        new_cols["Return"] = new_cols["Return"] * -1
		
    df = pd.concat([df, new_cols], axis=1)
    
    target = (df["Return"] >= threshold).astype(int)
	

    target = df[f'Return_{exposicion_dias}'].copy()
    target = (df[f'Return_{exposicion_dias}'] >= threshold).astype(int)
    df = pd.concat([df, target.rename("Target")], axis=1) 

    for i in range (4,31, 2):
        target = (df[f'Return_{i}'] >= threshold).astype(int)
        df = pd.concat([df, target.rename(f'Target_{i}')], axis=1) 
	
    

    df = df.copy()
    df[date_column] = pd.to_datetime(df[date_column], dayfirst=True)

    day_of_month = df[date_column].apply(lambda x: x.day)
    month = df[date_column].apply(lambda x: x.month)
    day_of_week = df[date_column].apply(lambda x: x.weekday())
    year = df[date_column].apply(lambda x: x.year)

    df[date_column] = df[date_column].dt.strftime('%d/%m/%Y %H:%M')

    new_columns = pd.concat([day_of_month.rename('day_of_month'), month.rename('month'), day_of_week.rename('day_of_week'), year.rename('year')], axis=1)
    df = pd.concat([df, new_columns], axis=1)

    return df

def split_data_validation(df, year_max_cut = '2022', year_min_cur= '2008'):
    data = df.query('year >= ' + year_max_cut).copy()
    df = df.query(year_min_cur+' < year <= 2022')
    df = df.reset_index(drop=True)
    return df, data

def map_creator(df):
    inicio = time.time()
    columns = df.columns
    no_sft_columns = [col for col in columns if 'sft' not in col]
    column_map = create_column_map(df, columns, no_sft_columns)
    fin = time.time()
    duracion = fin - inicio
    print("La duración del proceso fue de", duracion, "segundos")
    return column_map

def create_column_map(df, columns, no_sft_columns):
    column_map = {}
    rsi_columns = {col for col in columns if 'rsi_' in col}
    adx_columns = {col for col in columns if 'adx' in col}
    plus_di_columns = {col for col in columns if 'plus_di' in col}
    minus_di_columns = {col for col in columns if 'minus_di' in col}
    will_columns = {col for col in columns if 'willr' in col}
    sma_columns = {col for col in columns if 'sma' in col}
    mema_columns = {col for col in columns if 'mema' in col}
    ibs_columns = {col for col in columns if 'ibs_' in col}
    atr_columns = {col for col in columns if 'atr' in col}
    bbup_columns = {col for col in columns if 'bb_upper' in col}
    bbmid_columns = {col for col in columns if 'bb_middle' in col}
    bblow_columns = {col for col in columns if 'bb_lower' in col}
    macd_columns = {col for col in columns if 'macd' in col}
    macdsig_columns = {col for col in columns if 'macdsig' in col}
    macdh_columns = {col for col in columns if 'macdh' in col}
    ibsma_columns = {col for col in columns if 'ibma' in col}
    hh_columns = {col for col in columns if 'hh' in col}
    dayw_columns = {col for col in columns if 'day_of_week' in col}
    daym_columns = {col for col in columns if 'day_of_month' in col}
    ll_columns = {col for col in columns if 'll' in col}
    mom_columns = {col for col in columns if 'mom' in col}
    aaro_columns = {col for col in columns if 'aaro_' in col}
    roc_columns = {col for col in columns if 'roc' in col}
    stoch_columns = {col for col in columns if 'stoch' in col}
    stochk_columns = {col for col in columns if 'stochk' in col}
    stochd_columns = {col for col in columns if 'stochd' in col}
    stdev_columns = {col for col in columns if 'stdev' in col}
    aarod_columns = {col for col in columns if 'aarod_' in col}
    aarou_columns = {col for col in columns if 'aarou_' in col}
    
    for column in no_sft_columns:
        if 'rsi' in column:
            filtered_columns = rsi_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col and 'ibs' not in col}
            column_map[column] = [list(range(0, 101)), list(filtered_columns)]

        elif 'adx' in column:
            filtered_columns = adx_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(range(0, 101)), list(filtered_columns)]
        
        elif 'plus_di' in column:
            filtered_columns = plus_di_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(range(0, 101)), list(filtered_columns)]
                                                            
        elif 'minus_di' in column:
            filtered_columns = minus_di_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(range(0, 101)), list(filtered_columns)]
            
        elif 'willr' in column:
            filtered_columns = will_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(x for x in range(0, -101, -1)), list(filtered_columns)]
                                                            
        elif 'sma' in column:
            filtered_columns = sma_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(['Open', 'High', 'Low', 'Close', 'Close_sft_1', 'Close_sft_2', 'Close_sft_3', 'Low_sft_1', 'Low_sft_2', 'Low_sft_3', 'High_sft_1', 'High_sft_2', 'High_sft_3']), list(filtered_columns)]  

        elif 'mema' in column:
            filtered_columns = mema_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(['Open', 'High', 'Low', 'Close', 'Close_sft_1', 'Close_sft_2', 'Close_sft_3', 'Low_sft_1', 'Low_sft_2', 'Low_sft_3', 'High_sft_1', 'High_sft_2', 'High_sft_3']), list(filtered_columns)]			
            
        elif 'ibs_' in column:
            filtered_columns = ibs_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list([i / 100 for i in range(0, 101)]), list(filtered_columns)]
        
        elif 'atr' in column:
            filtered_columns = atr_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(filtered_columns)]
            
        elif 'bb_upper' in column:
            filtered_columns = bbup_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(['Open', 'High', 'Low', 'Close', 'Close_sft_1', 'Close_sft_2', 'Close_sft_3', 'Low_sft_1', 'Low_sft_2', 'Low_sft_3', 'High_sft_1', 'High_sft_2', 'High_sft_3']), list(filtered_columns)]
        
        elif 'bb_middle' in column:
            filtered_columns = bbmid_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(['Open', 'High', 'Low', 'Close', 'Close_sft_1', 'Close_sft_2', 'Close_sft_3', 'Low_sft_1', 'Low_sft_2', 'Low_sft_3', 'High_sft_1', 'High_sft_2', 'High_sft_3']), list(filtered_columns)]
            
        elif 'bb_lower' in column:
            filtered_columns = bblow_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(['Open', 'High', 'Low', 'Close', 'Close_sft_1', 'Close_sft_2', 'Close_sft_3', 'Low_sft_1', 'Low_sft_2', 'Low_sft_3', 'High_sft_1', 'High_sft_2', 'High_sft_3']), list(filtered_columns)]
            
        elif 'macd' in column:
            filtered_columns = macd_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(macdsig_columns)]
        
        elif 'macdsig' in column:
            filtered_columns = macdsig_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(macd_columns)]
                                  
        elif 'macdh' in column:
            filtered_columns = macdh_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(filtered_columns)]
        
        elif 'ibma' in column:
            filtered_columns = ibsma_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(ibs_columns)]
        
        elif 'hh' in column:
            filtered_columns = hh_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(['Open', 'High', 'Low', 'Close', 'Close_sft_1', 'Close_sft_2', 'Close_sft_3', 'Low_sft_1', 'Low_sft_2', 'Low_sft_3', 'High_sft_1', 'High_sft_2', 'High_sft_3'])]
            
        elif 'day_of_week' in column:
            filtered_columns = dayw_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list([i for i in range(0, 7)]), list([i for i in range(0, 7)])]
        
        elif 'day_of_week' in column:
            filtered_columns = dayw_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list([i for i in range(0, 7)]), list([i for i in range(0, 7)])]
        
        elif 'day_of_month' in column:
            filtered_columns = daym_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list([i for i in range(1, 32)]), list([i for i in range(1, 32)])]
        
        
        elif 'mom' in column:
            filtered_columns = mom_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(filtered_columns)]
            
        elif 'aaro_' in column:
            filtered_columns = aaro_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(x for x in range(-100, 101, 1)), list(filtered_columns)]
			
        elif 'aarod_' in column:
            filtered_columns = aarod_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(x for x in range(0, 101)), list(filtered_columns)]
			
        elif 'aarou_' in column:
            filtered_columns = aarou_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(x for x in range(0, 101)), list(filtered_columns)]
        
        elif 'roc' in column:
            filtered_columns = roc_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(filtered_columns)]
        
        elif 'stochd' in column:
            filtered_columns = stochd_columns - {column}
            comun_columns = stoch_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            filtered_comun_columns = {col for col in comun_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(filtered_comun_columns)]
            
        elif 'stochk' in column:
            filtered_columns = stochk_columns - {column}
            comun_columns = stoch_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            filtered_comun_columns = {col for col in comun_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(filtered_comun_columns)]
        
        elif 'stdev' in column:
            filtered_columns = stdev_columns - {column}
            filtered_columns = {col for col in filtered_columns if 'condition' not in col}
            column_map[column] = [list(filtered_columns), list(filtered_columns)]
            

    return column_map

In [3]:
def crear_directorio(nombre_carpeta):
    try:
        os.makedirs(nombre_carpeta)
        print(f"Directorio '{nombre_carpeta}' creado con éxito.")
    except FileExistsError:
        print(f"El directorio '{nombre_carpeta}' ya existe.")
    except Exception as e:
        print(f"Error al crear el directorio '{nombre_carpeta}': {e}")

In [4]:

date_column = 'DateTime'
h12 = 'GBPNZ_H12'


In [5]:
exposicion_dias=4
threshold=65
df = transform_df(h12, exposicion_dias, threshold, short=True)
dir_results_name = f'results_{exposicion_dias}_{threshold}'
crear_directorio(dir_results_name)
full_df = df.copy()
df = df.query('2016 < year < 2019')

df = df.reset_index(drop=True)
column_map = map_creator(df)



El directorio 'results_4_65' ya existe.
La duración del proceso fue de 0.020105361938476562 segundos


In [6]:
df.head()

,DateTime,Open,High,Low,Close,rsi_2,rsi_4,rsi_6,rsi_8,rsi_10,rsi_12,rsi_14,rsi_16,rsi_18,rsi_20,rsi_22,rsi_24,rsi_26,rsi_28,rsi_30,rsi_32,rsi_34,rsi_36,rsi_38,rsi_40,rsi_42,rsi_44,rsi_46,rsi_48,rsi_50,adx_2,adx_4,adx_6,adx_8,adx_10,adx_12,adx_14,adx_16,adx_18,adx_20,adx_22,adx_24,adx_26,adx_28,adx_30,adx_32,adx_34,adx_36,adx_38,adx_40,...,aaro_32_sft_1,aaro_32_sft_2,aaro_32_sft_3,adx_38_sft_1,adx_38_sft_2,adx_38_sft_3,rsi_26_sft_1,rsi_26_sft_2,rsi_26_sft_3,bb_upper_2_19_sft_1,bb_upper_2_19_sft_2,bb_upper_2_19_sft_3,aarou_36_sft_1,aarou_36_sft_2,aarou_36_sft_3,Return_2,Return_4,Return_6,Return_8,Return_10,Return_12,Return_14,Return_16,Return_18,Return_20,Return_22,Return_24,Return_26,Return_28,Return_30,Return,Target,Target_4,Target_6,Target_8,Target_10,Target_12,Target_14,Target_16,Target_18,Target_20,Target_22,Target_24,Target_26,Target_28,Target_30,day_of_month,month,day_of_week,year
0,02/01/2017 00:00,1.7762100,1.7822700,1.7759800,1.7809200,92.6545139,72.8057877,63.2533092,58.8690820,56.6882945,55.5141842,54.8484720,54.4591555,54.2251445,54.0779729,53.9772857,53.8989870,53.8287873,53.7584135,53.6832948,53.6011284,53.5109864,53.4127612,53.3068218,53.1938010,53.0744649,52.9496332,52.8201302,52.6867556,52.5502679,75.3424000,38.7045901,25.9484515,20.4548364,18.0911441,16.7187523,16.3078780,16.3435063,16.2278852,16.0147792,15.8045486,15.5619850,15.2054163,14.7828293,14.3371580,13.8961694,13.4880406,13.1006750,12.7753997,12.5205955,...,12.5000000,12.5000000,12.5000000,12.7949187,12.8266368,12.9843229,53.5153957,50.3896367,47.2977253,1.7963023,1.7974522,1.7991956,44.4444444,0.0000000,0.0000000,73.3000000,113.4000000,166.5000000,179.4000000,369.1000000,454.2000000,512.0000000,590.0000000,663.3000000,799.6000000,849.7000000,695.3000000,668.9000000,615.9000000,472.0000000,113.4000000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,0,2017
1,02/01/2017 12:00,1.7810000,1.7818000,1.7650800,1.7709700,29.0029709,44.1730368,45.7954452,46.4429826,47.0598530,47.6610710,48.2225646,48.7328290,49.1869246,49.5833598,49.9231053,50.2089542,50.4448807,50.6354652,50.7854555,50.8994747,50.9818518,51.0365405,51.0670961,51.0766845,51.0681094,51.0438466,51.0060794,50.9567328,50.8975041,61.7697450,33.7519466,23.8247929,19.1032032,16.9279272,15.6277764,15.2283330,15.3742483,15.4644344,15.4037139,15.3043203,15.1446141,14.8540141,14.4836689,14.0788611,13.6694624,13.2851317,12.9159460,12.6034613,12.3569993,...,12.5000000,12.5000000,12.5000000,12.7753997,12.7949187,12.8266368,53.8287873,53.5153957,50.3896367,1.7953675,1.7963023,1.7974522,41.6666667,44.4444444,0.0000000,20.1000000,28.4000000,31.3000000,60.2000000,378.5000000,292.1000000,405.0000000,577.0000000,616.1000000,758.1000000,511.0000000,500.4000000,546.6000000,449.8000000,384.2000000,28.4000000,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,2,1,0,2017
2,03/01/2017 00:00,1.7720700,1.7741100,1.7624900,1.7733900,46.7855336,50.4875853,49.8362802,49.4111384,49.3831641,49.5544670,49.8106341,50.0942022,50.3743290,50.6337975,50.8634612,51.0592998,51.2205318,51.3483227,51.4449109,51.5130407,51.5556110,51.5754724,51.5753175,51.5576310,51.5246725,51.4784792,51.4208780,51.3535017,51.2778064,61.5441949,32.4864850,23.2618640,18.6466277,16.3691152,14.9786461,14.4898420,14.5662474,14.6306190,14.6903562,14.7170320,14.6526050,14.4380572,14.1282653,13.7712663,13.3993238,13.0437986,12.6970800,12.4011041,12.1662297,...,12.5000000,12.5000000,12.5000000,12.6034613,12.7753997,12.7949187,50.4448807,53.8287873,53.5153957,1.7930105,1.7953675,1.7963023,38.8888889,41.6666667,44.4444444,38.1000000,91.2000000,104.1000000,293.8000000,378.9000000,436.7000000,514.7000000,588.0000000,724.3000000,774.4000000,620.0000000,593.6000000,540.6000000,396.7000000,478.3000000,91.2000000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,2017
3,03/01/2017 12:00,1.7733900,1.7786900,1.7637200,1.7687600,23.8895916,39.1820766,42.5532094,44.0706899,45.1692274,46.0755703,46.8499735,47.5190665,48.0974776,48.5947480,49.0185137,49.3758212,49.6734917,49.9180937,50.1158170,50.2723648,50.3928974,50.48202

In [7]:
len(df.columns)

1825

In [8]:
return_columns = [col for col in df.columns if 'Return_' in col]
df[return_columns+['Return']].tail(25)

,Return_2,Return_4,Return_6,Return_8,Return_10,Return_12,Return_14,Return_16,Return_18,Return_20,Return_22,Return_24,Return_26,Return_28,Return_30,Return
1013,-123.4000000,-70.2000000,10.8000000,-13.2000000,-297.8000000,-395.2000000,-357.3000000,-429.0000000,-341.6000000,-450.5000000,-500.4000000,-556.0000000,-525.8000000,-454.2000000,-457.9000000,-70.2000000
1014,-60.5000000,-76.7000000,4.3000000,-177.0000000,-220.6000000,-362.5000000,-414.6000000,-399.6000000,-379.6000000,-423.4000000,-528.0000000,-456.5000000,-487.8000000,-453.4000000,-416.2000000,-76.7000000
1015,51.2000000,132.2000000,108.2000000,-176.4000000,-273.8000000,-235.9000000,-307.6000000,-220.2000000,-329.1000000,-379.0000000,-434.6000000,-404.4000000,-332.8000000,-336.5000000,-373.8000000,132.2000000
1016,-18.2000000,62.8000000,-118.5000000,-162.1000000,-304.0000000,-356.1000000,-341.1000000,-321.1000000,-364.9000000,-469.5000000,-398.0000000,-429.3000000,-394.9000000,-357.7000000,-438.1000000,62.8000000
1017,79.0000000,55.0000000,-229.6000000,-327.0000000,-289.1000000,-360.8000000,-273.4000000,-382.3000000,-432.2000000,-487.8000000,-457.6000000,-386.0000000,-389.7000000,-427.0000000,-424.9000000,55.0000000
1018,79.0000000,-102.3000000,-145.9000000,-287.8000000,-339.9000000,-324.9000000,-304.9000000,-348.7000000,-453.3000000,-381.8000000,-413.1000000,-378.7000000,-341.5000000,-421.9000000,-350.7000000,-102.3000000
1019,-26.0000000,-310.6000000,-408.0000000,-370.1000000,-441.8000000,-354.4000000,-463.3000000,-513.2000000,-568.8000000,-538.6000000,-467.0000000,-470.7000000,-508.0000000,-505.9000000,-426.6000000,-310.6000000
1020,-183.3000000,-226.9000000,-368.8000000,-420.9000000,-405.9000000,-385.9000000,-429.7000000,-534.3000000,-462.8000000,-494.1000000,-459.7000000,-422.5000000,-502.9000000,-431.7000000,-360.0000000,-226.9000000
1021,-286.6000000,-384.0000000,-346.1000000,-417.8000000,-330.4000000,-439.3000000,-489.2000000,-544.8000000,-514.6000000,-443.0000000,-446.7000000,-484.0000000,-481.9000000,-402.6000000,-367.1000000,-384.0000000
1022,-45.6000000,-187.5000000,-239.6000000,-224.6000000,-204.6000000,-248.4000000,-353.0000000,-281.5000000,-312.8000000,-278.4000000,-241.2000000,-321.6000000,-250.4000000,-178.7000000,-33.8000000,-187.5000000


In [9]:
print(len(df[df['day_of_week'] == 6]))
print(len(df[df['day_of_week'] == 7]))

0
0


In [10]:
def generate_all_rules(column_map, df):
    all_rules = [] 
    
    for column, (possible_values, related_columns) in column_map.items():
        if 'day' in column:
            operators = ['>', '<', '==', '>=', '<=']
        else:
            operators = ['>=', '<=']

        for value in possible_values:
            for operator in operators:
                condition = f"{column} {operator} {value}"
                all_rules.append(condition)

        for value in related_columns:
                for operator in operators:
                    related_condition = f"{column} {operator} {value}"
                    all_rules.append(related_condition)
    
    return all_rules




In [11]:
all_rules = generate_all_rules(column_map, df)

In [12]:
all_rules

['rsi_2 >= 0',
 'rsi_2 <= 0',
 'rsi_2 >= 1',
 'rsi_2 <= 1',
 'rsi_2 >= 2',
 'rsi_2 <= 2',
 'rsi_2 >= 3',
 'rsi_2 <= 3',
 'rsi_2 >= 4',
 'rsi_2 <= 4',
 'rsi_2 >= 5',
 'rsi_2 <= 5',
 'rsi_2 >= 6',
 'rsi_2 <= 6',
 'rsi_2 >= 7',
 'rsi_2 <= 7',
 'rsi_2 >= 8',
 'rsi_2 <= 8',
 'rsi_2 >= 9',
 'rsi_2 <= 9',
 'rsi_2 >= 10',
 'rsi_2 <= 10',
 'rsi_2 >= 11',
 'rsi_2 <= 11',
 'rsi_2 >= 12',
 'rsi_2 <= 12',
 'rsi_2 >= 13',
 'rsi_2 <= 13',
 'rsi_2 >= 14',
 'rsi_2 <= 14',
 'rsi_2 >= 15',
 'rsi_2 <= 15',
 'rsi_2 >= 16',
 'rsi_2 <= 16',
 'rsi_2 >= 17',
 'rsi_2 <= 17',
 'rsi_2 >= 18',
 'rsi_2 <= 18',
 'rsi_2 >= 19',
 'rsi_2 <= 19',
 'rsi_2 >= 20',
 'rsi_2 <= 20',
 'rsi_2 >= 21',
 'rsi_2 <= 21',
 'rsi_2 >= 22',
 'rsi_2 <= 22',
 'rsi_2 >= 23',
 'rsi_2 <= 23',
 'rsi_2 >= 24',
 'rsi_2 <= 24',
 'rsi_2 >= 25',
 'rsi_2 <= 25',
 'rsi_2 >= 26',
 'rsi_2 <= 26',
 'rsi_2 >= 27',
 'rsi_2 <= 27',
 'rsi_2 >= 28',
 'rsi_2 <= 28',
 'rsi_2 >= 29',
 'rsi_2 <= 29',
 'rsi_2 >= 30',
 'rsi_2 <= 30',
 'rsi_2 >= 31',
 'rsi_2 <= 3

In [13]:
len(all_rules)

251680

In [14]:


def generate_hash(s):
    try:
        first_part = re.findall(r'^\D+', s.split(' ')[0])[0]
        comparison_operator = re.findall(r'[<=>=]+', s)[0]
        try:
            second_part = re.findall(r'^\D+', s.split(' ')[2].split('_')[0])[0]
        except IndexError:
            second_part = 'num'
        combined = first_part + comparison_operator + second_part
        return hashlib.sha256(combined.encode()).hexdigest()
    except Exception as e:
        return 12345678910


def process_rule_chunk(df, data, df_columns, rule_chunk, target_values, target, returns_columns):
    chunk_results = {}
    chunk_stats = {}
    for rule in rule_chunk:
        try:
            parts = rule.split()
            if len(parts) == 3:
                column1, operator, column2_or_value = parts
                idx1 = df_columns.get_loc(column1)
                idx_return = df_columns.get_loc('Return')
                
                try:
                    value = float(column2_or_value)
                    idx2 = None
                except ValueError:
                    idx2 = df_columns.get_loc(column2_or_value)
                    
                condition_eval_df = None
                if operator == '>=':
                    if idx2 is None:
                        condition_eval_df = (data[:, idx1] >= value)
                    else:
                        condition_eval_df = (data[:, idx1] >= data[:, idx2])
                elif operator == '<=':
                    if idx2 is None:
                        condition_eval_df = (data[:, idx1] <= value)
                    else:
                        condition_eval_df = (data[:, idx1] <= data[:, idx2])
                elif operator == '==':
                    if idx2 is None:
                        condition_eval_df = (data[:, idx1] == value)
                    else:
                        condition_eval_df = (data[:, idx1] == data[:, idx2])
                        
                condition_eval = condition_eval_df.astype(np.int8)
                ones_count = np.sum(condition_eval)
                if ones_count < 100:
                    continue
                
                correlation, _ = pointbiserialr(condition_eval, target)
                if np.isnan(correlation):
                    continue

                length = len(condition_eval)
                zeros_count = length - ones_count
                win_rate = np.sum(condition_eval & target_values) / ones_count if ones_count > 0 else 0

                
                sum_returns = np.sum(data[condition_eval_df, idx_return])
                
                sum_positive_returns = np.sum(data[condition_eval_df & (data[:, idx_return] > 0), idx_return])
                sum_negative_returns = np.sum(data[condition_eval_df & (data[:, idx_return] < 0), idx_return])
                
                profit_factor = 0
                if sum_negative_returns != 0:
                    profit_factor = sum_positive_returns / -sum_negative_returns
                else:
                    profit_factor = float('inf')
                    
                optimal_pf = 0
                optimal_exposition = str(4)
                profit_factors = {}
                for ret_col in returns_columns:
                    idx_return = df_columns.get_loc(ret_col)
                    sum_positive_returns = np.sum(data[condition_eval_df & (data[:, idx_return] > 0), idx_return])
                    sum_negative_returns = -np.sum(data[condition_eval_df & (data[:, idx_return] < 0), idx_return])
                    profit_factor = sum_positive_returns / sum_negative_returns if sum_negative_returns != 0 else float('inf')
                    match = re.search(r'Return_(\d+)', ret_col)
                    number = 4
                    if match:
                        number = match.group(1)
                        chunk_stats[f'pf_{str(number)}'] = profit_factor
                        if(profit_factor != np.inf and profit_factor > optimal_pf):
                            optimal_pf = profit_factor
                            optimal_exposition = str(number)
                        
                target_optimal = df[f'Target_{optimal_exposition}'].values
                target_values_optimal = df[f'Target_{optimal_exposition}'].apply(lambda x: 1 if x > 0 else 0).values
                correlation_optimal, _ = pointbiserialr(condition_eval, target_optimal)
                win_rate_optimal = np.sum(condition_eval & target_values_optimal) / ones_count if ones_count > 0 else 0
                
                
                chunk_results[rule] = condition_eval
                chunk_stats[rule] = {
                    'correlation': correlation,
                    'length': length,
                    'ones_count': ones_count,
                    'zeros_count': zeros_count,
                    'win_rate': round(win_rate * 100, 2),
                    'return': sum_returns,
                    'hash': generate_hash(rule),
                    'optimal_exposition': optimal_exposition,
                    'correlation_optimal': correlation_optimal,
                    'win_rate_optimal': round(win_rate_optimal * 100, 2),
                }
                chunk_stats[rule].update({f'pf_{i}': chunk_stats.pop(f'pf_{i}') for i in range(4, 31, 2)})
                
            else:
                print(f"Rule '{rule}' could not be parsed.")
        except Exception as e:
            print(f"Error processing rule '{rule}': {e}")

    return chunk_results, chunk_stats


def evaluate_rules_numpy(df, all_rules, target_column='Target', output_file='results2_h5.h5'):
    print('Starting process..', datetime.now().strftime('%d-%m-%Y %H:%M:%S'))
    data = df.to_numpy()
    target = df[target_column].values
    target_values = df[target_column].apply(lambda x: 1 if x > 0 else 0).values


    num_cores = joblib.cpu_count()

    # Dividimos las reglas en chunks
    chunk_size = len(all_rules) // num_cores 
    rule_chunks = [all_rules[i:i + chunk_size] for i in range(0, len(all_rules), chunk_size)]
    
    returns_columns = [f'Return_{i}' for i in range(4, 31, 2)]  

    # Utilizar Joblib para paralelizar la evaluación de las reglas
    results = Parallel(n_jobs=-1)(
        delayed(process_rule_chunk)(df, data, df.columns, rule_chunk, target_values, target, returns_columns) for rule_chunk in rule_chunks
    )
    
    all_results = {}
    all_stats = {}
    for chunk_results, chunk_stats in results:
        all_results.update(chunk_results)
        all_stats.update(chunk_stats)
        
    result_df = pd.DataFrame(all_results)
    result_df['Target'] = df['Target'].copy()
    result_df['Return'] = df['Return'].copy()
    
    returns_columns = []
    for i in range(4, 31, 2):
        returns_columns.append(f'Return_{i}')
        
        
    for column_ in returns_columns:
        result_df[column_] = df[column_].values
        
    
    print('Ending process.. saving', datetime.now().strftime('%d-%m-%Y %H:%M:%S'), len(result_df))
    result_df.to_hdf(output_file, key='result_df', mode='w')
    sorted_stats = sorted(all_stats.items(), key=lambda item: item[1]['correlation'], reverse=True)
    
    groups = groupby(sorted_stats, key=lambda item: item[1]['correlation'])
    unique_stats = [next(g) for _, g in groups]
    
    df = pd.DataFrame([item[1] for item in unique_stats])
    df['condition'] = [item[0] for item in unique_stats]
    df = df[['condition'] + [col for col in df.columns if col != 'condition']]
    
    print('Ended process',datetime.now().strftime('%d-%m-%Y %H:%M:%S'))
    return df, unique_stats



In [15]:
start_time = time.time()
df_rules, sorted_stats = evaluate_rules_numpy(df, all_rules)

print(f"El proceso tardó {time.time() - start_time} segundos.")


Starting process.. 05-03-2025 18:30:53
Ending process.. saving 05-03-2025 18:32:51 1038
Ended process 05-03-2025 18:32:56
El proceso tardó 123.88965678215027 segundos.


In [19]:
len(df_rules)

94841

In [16]:
df_rules

,condition,correlation,length,ones_count,zeros_count,win_rate,return,hash,optimal_exposition,correlation_optimal,win_rate_optimal,pf_4,pf_6,pf_8,pf_10,pf_12,pf_14,pf_16,pf_18,pf_20,pf_22,pf_24,pf_26,pf_28,pf_30
0,sma_132 >= sma_136,0.1759592,1038,672,366,36.0100000,3745.3000000,724f0edc848b1caa180dcda4bf9f5a94c2dcd0eddd8745...,30,0.2573752,43.0100000,1.0945988,1.1335547,1.1613260,1.1840314,1.1943398,1.2041190,1.2017112,1.1944720,1.2152646,1.2459454,1.2718313,1.2987201,1.3215607,1.3391664
1,sma_154 >= sma_156,0.1738629,1038,681,357,35.8300000,6173.2000000,724f0edc848b1caa180dcda4bf9f5a94c2dcd0eddd8745...,10,0.2096595,43.3200000,1.1658553,1.2382594,1.2671310,1.2939667,1.2836159,1.2754324,1.2668587,1.2524731,1.2421759,1.2463313,1.2524315,1.2642861,1.2663413,1.2801791
2,sma_130 >= sma_136,0.1735330,1038,674,364,35.9100000,3744.0000000,724f0edc848b1caa180dcda4bf9f5a94c2dcd0eddd8745...,30,0.2803697,43.7700000,1.0947644,1.1172121,1.1452505,1.1734997,1.2015198,1.2084156,1.2115714,1.2080466,1.2216756,1.2499496,1.2767179,1.3070396,1.3300607,1.3468401
3,sma_128 >= sma_136,0.1735330,1038,674,364,35.9100000,3790.1000000,724f0edc848b1caa180dcda4bf9f5a94c2dcd0eddd8745...,30,0.2931549,44.2100000,1.0963892,1.1195027,1.1424985,1.1762149,1.1995262,1.2191539,1.2249998,1.2123961,1.2311692,1.2518091,1.2763017,1.3080222,1.3300554,1.3459047
4,sma_152 >= sma_158,0.1697904,1038,688,350,35.6100000,6244.2000000,724f0edc848b1caa180dcda4bf9f5a94c2dcd0eddd8745...,30,0.2539142,42.5900000,1.1667699,1.2160523,1.2491966,1.2661370,1.2818561,1.2856327,1.2733315,1.2552954,1.2551985,1.2508305,1.2612209,1.2748566,1.2862542,1.2970250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94836,sma_152 <= sma_158,-0.1697904,1038,350,688,19.1400000,-13048.9000000,9620e21a74658492f7af120824bf7d2062ca3a47b721c9...,4,-0.1697904,19.1400000,0.4772306,0.4236770,0.3941917,0.3721503,0.3405120,0.3178057,0.3117340,0.3053370,0.2813863,0.2461886,0.2213764,0.1968827,0.1773236,0.1602976
94837,sma_128 <= sma_136,-0.1735330,1038,364,674,19.2300000,-10594.8000000,9620e21a74658492f7af120824bf7d2062ca3a47b721c9...,4,-0.1735330,19.2300000,0.5409989,0.4939464,0.4431775,0.3869652,0.3409673,0.3107379,0.2933963,0.2777139,0.2548681,0.2265148,0.1977361,0.1722056,0.1523450,0.1378027
94838,sma_130 <= sma_136,-0.1735330,1038,364,674,19.2300000,-10548.7000000,9620e21a74658492f7af120824bf7d2062ca3a47b721c9...,4,-0.1735330,19.2300000,0.5392494,0.4927477,0.4374295,0.3892490,0.3393157,0.3171916,0.3048161,0.2835595,0.2626991,0.2331277,0.2032743,0.1794314,0.1579479,0.1423217
94839,sma_154 <= sma_156,-0.1738629,1038,357,681,19.0500000,-12977.9000000,9620e21a74658492f7af120824bf7d2062ca3a47b721c9...,4,-0.1738629,19.0500000,0.4846502,0.4174703,0.3925475,0.3642899,0.3486141,0.3307713,0.3202912,0.3173048,0.3026194,0.2638296,0.2357592,0.2124027,0.1909470,0.1736078


In [17]:
sorted_stats

[('sma_132 >= sma_136',
  {'correlation': 0.17595923240178807,
   'length': 1038,
   'ones_count': 672,
   'zeros_count': 366,
   'win_rate': 36.01,
   'return': 3745.2999999999984,
   'hash': '724f0edc848b1caa180dcda4bf9f5a94c2dcd0eddd874575510f7c410089d747',
   'optimal_exposition': '30',
   'correlation_optimal': 0.2573751630846861,
   'win_rate_optimal': 43.01,
   'pf_4': 1.0945988270179894,
   'pf_6': 1.133554701828222,
   'pf_8': 1.1613260086421269,
   'pf_10': 1.1840314006061512,
   'pf_12': 1.194339793196536,
   'pf_14': 1.2041189563016537,
   'pf_16': 1.2017112158106695,
   'pf_18': 1.1944720249157719,
   'pf_20': 1.2152645500150872,
   'pf_22': 1.2459454401796488,
   'pf_24': 1.2718312695396665,
   'pf_26': 1.2987201028253736,
   'pf_28': 1.3215607383086458,
   'pf_30': 1.3391663651973096}),
 ('sma_154 >= sma_156',
  {'correlation': 0.17386288895856594,
   'length': 1038,
   'ones_count': 681,
   'zeros_count': 357,
   'win_rate': 35.83,
   'return': 6173.199999999999,
   'ha

In [18]:
print(f'Total de backtests ejecutados: {len(df_rules) * 14}')

Total de backtests ejecutados: 1327774


In [20]:
df_sorted = df_rules.sort_values(by='pf_10', ascending=False)

In [21]:
df_sorted

,condition,correlation,length,ones_count,zeros_count,win_rate,return,hash,optimal_exposition,correlation_optimal,win_rate_optimal,pf_4,pf_6,pf_8,pf_10,pf_12,pf_14,pf_16,pf_18,pf_20,pf_22,pf_24,pf_26,pf_28,pf_30
8438,adx_10 >= 45,0.0683551,1038,101,937,39.6000000,4259.6000000,4118d96628e9d7109ca1b25b71d8f7665de5864ad974c7...,14,0.2609333,75.2500000,2.1586965,3.1715942,5.2336399,5.8244613,6.9239599,6.9773091,6.0108345,6.2502094,5.2758750,3.4241981,2.4650404,1.9297591,1.6192509,1.3581585
16426,adx_8 >= 49,0.0476905,1038,119,919,36.1300000,3932.6000000,4118d96628e9d7109ca1b25b71d8f7665de5864ad974c7...,14,0.2512457,70.5900000,1.8636244,2.3164482,3.3545015,4.3181896,5.6846869,5.8015489,5.4258785,5.6936880,5.3856474,4.1713673,2.8461556,2.3138205,1.8549034,1.5376714
6925,adx_10 >= 44,0.0740708,1038,119,919,39.5000000,5049.7000000,4118d96628e9d7109ca1b25b71d8f7665de5864ad974c7...,8,0.0864358,46.2200000,2.2195870,2.8858108,4.2723283,4.1302967,4.0078247,3.8836694,4.0282047,4.1335836,3.6197125,2.7507556,2.0954383,1.7085709,1.4721115,1.2493729
20154,adx_8 >= 50,0.0403423,1038,107,931,35.5100000,3154.7000000,4118d96628e9d7109ca1b25b71d8f7665de5864ad974c7...,12,0.1679375,59.8100000,1.7349159,2.0697175,3.0963659,4.0353760,5.4678409,5.2750235,5.1160721,5.0887923,4.9259333,3.6028435,2.4750864,1.9581171,1.6038102,1.3184407
13071,adx_8 >= 47,0.0550907,1038,146,892,36.3000000,4245.1000000,4118d96628e9d7109ca1b25b71d8f7665de5864ad974c7...,18,0.2556472,69.1800000,1.7321536,2.2396108,2.7646575,3.2718674,3.5791992,3.5967158,3.8912657,4.4281906,4.0536681,3.6055858,2.6849322,2.2312811,1.8815706,1.6341762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92674,adx_32 <= 10,-0.1042059,1038,115,923,16.5200000,-4318.5000000,3c92cb65188a6807549ad02f373e5063a01a53cbce3d2b...,4,-0.1042059,16.5200000,0.4360357,0.3515219,0.2741296,0.1942094,0.1452069,0.1365817,0.1209620,0.0973615,0.1046053,0.1025038,0.0911349,0.0787169,0.0733734,0.0758477
93255,adx_28 <= 11,-0.1114787,1038,132,906,16.6700000,-4835.2000000,3c92cb65188a6807549ad02f373e5063a01a53cbce3d2b...,4,-0.1114787,16.6700000,0.4454664,0.3423318,0.2578783,0.1939875,0.1349073,0.1273492,0.1115631,0.0890225,0.0821544,0.0836435,0.0751936,0.0689191,0.0633438,0.0642930
94228,adx_34 <= 10,-0.1299728,1038,161,877,16.1500000,-5662.7000000,3c92cb65188a6807549ad02f373e5063a01a53cbce3d2b...,4,-0.1299728,16.1500000,0.4483649,0.3509221,0.2782188,0.1917709,0.1600584,0.1702510,0.1598318,0.1460361,0.1502482,0.1579491,0.1528828,0.1477426,0.1476500,0.1373021
91223,adx_38 <= 9,-0.0908279,1038,123,915,18.7000000,-4497.6000000,3c92cb65188a6807549ad02f373e5063a01a53cbce3d2b...,4,-0.0908279,18.7000000,0.4566935,0.3456602,0.2671753,0.1774894,0.1393128,0.1458444,0.1278517,0.1215267,0.1392646,0.1478815,0.1363607,0.1212123,0.1096359,0.1110809
